In [53]:
import requests
import pandas as pd

# API URL 설정
api_url = "http://openapi.airgwangsan.kr:8080/Gwangsan/getDustDataAPI?apiId=01"

# API 호출
response = requests.get(api_url)
# JSON 데이터를 가져오기
data = response.json()

In [ ]:
data

In [55]:
df = pd.DataFrame(data['response']['items'])
df_unique = df.drop_duplicates(subset=['place'])
df_unique

,place,PM1,PM10,PM2_5,CO2,TEMPERATURE,HUMIDITY,LATITUDE,LONGITUDE,COLLECTION_DATE
0,하남동 홈플러스 하남점,9.4,18.0,10.8,500.0,32.8,35.0,35.17861940,126.8066483,2024-06-18 21:30:59.0
132,소촌동 현성 오토텍,16.0,18.0,17.2,243.0,41.4,20.5,35.15502930,126.7882538,2024-06-18 21:25:59.0
261,신창동 해피키즈 어린이집,8.6,31.8,9.8,2108.0,34.8,40.8,35.19879315,126.8379488,2024-06-18 21:25:59.0
385,운남동 한사랑병원,7.2,18.0,8.6,422.0,40.4,22.3,35.17269000,126.8176300,2024-06-18 21:30:59.0
512,수완동 하바드 어린이집,8.0,21.6,9.6,419.0,38.5,24.9,35.20105000,126.8273400,2024-06-18 21:25:59.0
...,...,...,...,...,...,...,...,...,...,...
15768,월곡동 광산구 노인 복지관,7.4,23.0,9.0,378.0,40.2,23.1,35.17160030,126.8146973,2024-06-18 21:30:59.0
15884,복룡동 관두 경로당,14.1,22.0,16.1,413.0,37.5,26.6,35.10205000,126.7821600,2024-06-18 21:25:59.0
16013,신촌동 공항역,8.0,16.0,9.5,420.0,25.6,50.8,35.11038500,126.8819770,2024-06-18 21:25:59.0
16238,월계동 KB 국민은행,6.0,19.2,7.2,1006.0,36.1,42.8,35.21361160,126.8458252,2024-06-18 19:45:59.0


In [56]:
df_unique.rename(columns={'LATITUDE':'lat', 'LONGITUDE':'lot'})

,place,PM1,PM10,PM2_5,CO2,TEMPERATURE,HUMIDITY,lat,lot,COLLECTION_DATE
0,하남동 홈플러스 하남점,9.4,18.0,10.8,500.0,32.8,35.0,35.17861940,126.8066483,2024-06-18 21:30:59.0
132,소촌동 현성 오토텍,16.0,18.0,17.2,243.0,41.4,20.5,35.15502930,126.7882538,2024-06-18 21:25:59.0
261,신창동 해피키즈 어린이집,8.6,31.8,9.8,2108.0,34.8,40.8,35.19879315,126.8379488,2024-06-18 21:25:59.0
385,운남동 한사랑병원,7.2,18.0,8.6,422.0,40.4,22.3,35.17269000,126.8176300,2024-06-18 21:30:59.0
512,수완동 하바드 어린이집,8.0,21.6,9.6,419.0,38.5,24.9,35.20105000,126.8273400,2024-06-18 21:25:59.0
...,...,...,...,...,...,...,...,...,...,...
15768,월곡동 광산구 노인 복지관,7.4,23.0,9.0,378.0,40.2,23.1,35.17160030,126.8146973,2024-06-18 21:30:59.0
15884,복룡동 관두 경로당,14.1,22.0,16.1,413.0,37.5,26.6,35.10205000,126.7821600,2024-06-18 21:25:59.0
16013,신촌동 공항역,8.0,16.0,9.5,420.0,25.6,50.8,35.11038500,126.8819770,2024-06-18 21:25:59.0
16238,월계동 KB 국민은행,6.0,19.2,7.2,1006.0,36.1,42.8,35.21361160,126.8458252,2024-06-18 19:45:59.0


In [57]:
def pm_to_color(pm10, pm2_5):
    if pm10 >= 151 and pm2_5 >= 76:
        return 'red'
    elif pm10 >= 81 and pm2_5 >= 36:
        return 'yellow'
    elif pm10 >= 31 and pm2_5 >= 16:
        return 'green'
    else:
        return 'blue'

In [59]:
import folium
import pandas as pd

# 지도의 초기 위치 설정
mymap = folium.Map(location=[35.17861940, 126.8066483], zoom_start=12)

# 각 좌표에 대해 마커를 지도에 추가합니다.
for index, row in df_unique.iterrows():
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']
    place = row['place']
    pm10 = float(row['PM10'])
    pm2_5 = float(row['PM2_5'])

        
    if pm10 >= 151 and pm2_5 >= 76:
        folium.Marker(
            location=[latitude, longitude],
            popup=place,
            icon=folium.Icon(color='red', icon='cloud')
        ).add_to(mymap)
    elif pm10 >= 81 and pm2_5 >= 36:
            folium.Marker(
            location=[latitude, longitude],
            popup=place,
            icon=folium.Icon(color='orange', icon='cloud')
        ).add_to(mymap)
    elif pm10 >= 31 and pm2_5 >= 16:
            folium.Marker(
            location=[latitude, longitude],
            popup=place,
            icon=folium.Icon(color='green', icon='cloud')
        ).add_to(mymap)
    else:
            folium.Marker(
            location=[latitude, longitude],
            popup=place,
            icon=folium.Icon(color='blue', icon='cloud')
        ).add_to(mymap)
      


# 지도 출력
mymap



/var/folders/37/4x_nf3rj0cq34syjpm8jpwmc0000gn/T/ipykernel_4487/1337070484.py:26: UserWarning: color argument of Icon should be one of: {'lightred', 'pink', 'lightgreen', 'blue', 'beige', 'black', 'purple', 'darkgreen', 'lightgray', 'lightblue', 'green', 'red', 'darkred', 'gray', 'darkblue', 'white', 'darkpurple', 'cadetblue', 'orange'}.
  icon=folium.Icon(color='yellow', icon='cloud')
